<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [37]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as matplot
import itertools
import math
import sys
import os
import utils

module_path = os.path.abspath(os.path.join('../python/python_src_preprocessing/src'))

if module_path not in sys.path:
    sys.path.append(module_path)

from rf.regression import multiple_forests as multiple

from utils.reader.densitymap import load_directory
from tqdm import tqdm_notebook, tqdm
from ipywidgets import *

%load_ext autoreload
%autoreload 1
%aimport utils.reader.densitymap, rf.regression
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# setup
simulation = 'check/csvs/obstacle/'
experiment = 'check/csvs/experiment/'

number_of_targets = 3

In [33]:
# load training (simulation data)
train = load_directory(simulation, file_filter=lambda x: 'reduced' in x)

# load test (experiment data)
test = load_directory(experiment, file_filter=lambda x: 'filtered' in x)

loading directory check/csvs/obstacle/: 100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 33.01it/s]


check/csvs/obstacle/ (7023, 230)


loading directory check/csvs/experiment/: 100%|██████████████████████████████████████████| 7/7 [00:00<00:00, 18.13it/s]


check/csvs/experiment/ (6678, 230)


In [70]:
# run
regressors, \
normed_prediction, \
(score_training, score_test, score_oob), \
errors = multiple( \
    {'samples': train[0], 'targets': train[1]}, \
    {'samples': test[0], 'targets': test[1]}, \
    number_of_trees=50)

3



Learning:  33%|████████████████████████▋                                                 | 1/3 [00:00<00:00,  5.26it/s]

KeyboardInterrupt: 

In [68]:
print(errors['mean_abs_error'])

[0.45018219 0.19859097 0.4942324 ]
